## Gait Video Study 
### Validating the estimates 3D poses via CoP computed on the treadmill 
### This code qualitatively and quantitatively validates the center of mass trajectory of estimates body coordinates with their respective treadmill extracted CoP's on a frame by frame basis

We will do qualitative and quantitative validation for CoP. 
* First, we need to align the treadmill GaitCycles.csv file to the video time. This will help us align the corresponding video frames to the gait events i.e. HSR/HSL/TOR and TOL to be specific. 
* Once we know for each video, which frame numbers correspond to heel strikes and toe offs, we compute the sequence of frames that are in single support left phase, frames that are in single support right phase and similarly in double support phase. Thus, each frame of a video is labelled to be in SSR, SSL or DS phase. 
* Now, for each frame in DS phase, use the computed real world x, y coordinates for big toe, small toe and heel to make 2 triangular regions for both left and right feet, since in this phase, both feet are on ground and thus impact the center of pressure. Now, plot the corresponding actual COPX, COPY coordinate (as a red dot) for this particular frame. If this red dot lies in the shaded region of computed CoP drawn, we are good to claim that actual CoP lies in the approximate computed CoP region. Similarly, for each frame in the SSL phase, since left foot rests on ground for this phase, the CoP must be determined using the left foot, and hence use the computed x, y coordinates of the left big toe, small toe and heel to draw a shaded triangular region spanned by CoP for this frame, and draw the red dot for the actual x, y of CoP for this frame and if it lies within the shaded region, we are good to claim that actual and computed CoP region match. Now, for each frame in SSR phase, the shaded CoP region must be determined using the right feet's big/small toe and heel's x, y coordinates and if the actual CoP's x, y is bounded in this shaded computed CoP's region, we are good. 

* For qualitative validation, we plot these above mentioned regions for computed CoP and actual CoPs as markers in/out of that region for each video. If we do this for a complete stride, it should follow a butterfly pattern. And hence the inverted triangles and hexagons should occur in a butterfly pattern.
* For quantitative validation, we will call it success (1) if the actual CoP is bounded by the computed CoP shaded region and failure (0) otherwise for every frame of every video for every trial and cohort. 
* Further, for more precise quantitative validation, we can find the lateral, anterior-posterior and euclidean distance of the actual (COP_x, COP_y) with the centroid of our region drawn. This gives us a numerical value quantifying the error in the true and predicted CoP. This step can especially be done for only wrongly predicted values, to further check what is the measure of wrongly predicted values. Further, we may check that we may have error most in the lateral direction or most in the AP direction or eucliean only. 
* Based on the statistics of these success and failure counts, we can quantify the performance of our marker estimation framework using CoP validation. 
* Further, we can try to correlate/have a look at the distribution pattern to relate the correctness of CoP (either quantified using binary scores or using the numerical scores) with the confidence scores predicted by the OpenPose algorithm. Now, since we are only using toes and heel coordinates to draw the CoM trajectory/region, we should only use the confidence scores for heel and toes for this correlation. To be precise, we can average the confidence scores of left feet's heel and 2 toes to get the aggregated confidence score for frames in left single support, similarly, we can average the confidence scores of right feet's heel and 2 toes to get the aggregated confidence score for frames in right single support, and average the confidence scores of both the left and right feet's heel and toes to get the aggregated confidence scores for frames in double support. Now this correlation/relation between the confidence score for each frame and it's coorsponding correctness of CoP metric can be either done on a frame by frame basis. Or rather we can aggreagte all frames over a stride and do the relationship analysis on a stride by stride basis based on some aggregatd confidence scores of stride with some aggregated correctness of CoP score over each stride. 

In [ ]:
from importlib import reload
import imports 
reload(imports)
from imports import *

In [ ]:
#Folder for CoP validation
cop_path = 'C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\CoPvalidation\\'
#Path to log file corresponding to the sync files between frame numbers and treadmill identified events for each video
sync_log_file = cop_path + 'treamill_video_cop_sync.csv'

### Utility functions

### main()